In [57]:
%matplotlib inline
import seaborn
import quandl
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from bokeh.plotting import figure, show
import bokeh.charts as bok
from bokeh.io import output_notebook
import math
token="Us3wFmXGgAj_1cUtHAAR"

import pylab
pylab.rcParams['figure.figsize'] = (12, 8)
output_notebook()

Loading BokehJS ...

In [7]:
def tsmom(data,months):
    vol=pd.rolling_std(data.pct_change(),24)*math.sqrt(12)
    signal=data/data.shift(months)-1
    signal = signal /abs(signal)
    position=signal / vol 
    return position

def calc_pnl(position,data):
    return position*data.pct_change().shift(1)

def calc_Sharpe(pnl,N=12):
    return np.sqrt(N) * pnl.mean() / pnl.std()

def ew_portfolio_pnl(pnl):
    return pnl.divide(pnl.count(axis=1),axis=0).sum(axis=1)

In [8]:
data=pd.read_csv('liquid_contracts.csv',index_col=0,parse_dates=['Date']).resample(rule='m',how='last')
markets=pd.read_csv('contracts.csv',index_col=0)
volume=pd.read_csv('volume_data.csv',index_col=0,parse_dates=['Date']).resample(rule='m',how='sum')
fx=pd.read_csv('currency.csv',index_col=0,parse_dates=['DATE'])

/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  if __name__ == '__main__':
/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  app.launch_new_instance()


In [9]:
data=data.dropna(axis=1)

In [34]:
pos=tsmom(data,12)
pnl=calc_pnl(pos,data)
sr=calc_Sharpe(pnl)

/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: FutureWarning: pd.rolling_std is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=24,center=False).std()
  from IPython.kernel.zmq import kernelapp as app


In [39]:
show(bok.Bar(sr))

In [42]:
show(bok.TimeSeries(pnl.cumsum()))

In [51]:
df.head()

,SR,Volume,Sector
ES,1.046789,NaN,NaN
S&P MidCap 400,1.002407,8.710445e+07,Equities
DAX,1.014103,2.780778e+09,Equities
S&P 500,1.047525,3.614584e+10,Equities
Brent Crude,0.965645,1.685337e+08,Commodities


In [44]:
df=pd.DataFrame()
df['SR']=sr
df['Volume']=(volume*data).mean()
s=pd.Series()
for x in df.T.columns:
    try:
        s[x]=markets.T[x].Sector      
    except:
        print x
df['Sector']=s

ES


In [61]:
tooltips=[('Volume','@Volume'),('Sharpe Ratio','@SR')]
p= bok.Scatter(df,x='Volume',y='SR',color='Sector',
               tooltips=tooltips,title='Volume versus Sharpe Ratio')
show(p)